In [ ]:
import pandas as pd
import transformers

In [ ]:
df=pd.read_csv("https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv",delimiter="\t",header=None)

In [ ]:
df.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [ ]:
model_class,tokenizer_class,pretrained_weights=(transformers.DistilBertModel,transformers.DistilBertTokenizer,'distilbert-base-uncased')

## tokenize the sentences and coverting into encodings


In [ ]:
tokensizer=tokenizer_class.from_pretrained(pretrained_weights)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import torch
print(torch.cuda.is_available())


False


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')




In [ ]:
model=model_class.from_pretrained(pretrained_weights).to(device)

In [ ]:
df["tokenized"]=df[0].apply(lambda x:tokensizer.encode(x,add_special_tokens=True))

In [ ]:
df.head(4)

,0,1,tokenized
0,"a stirring , funny and finally transporting re...",1,"[101, 1037, 18385, 1010, 6057, 1998, 2633, 182..."
1,apparently reassembled from the cutting room f...,0,"[101, 4593, 2128, 27241, 23931, 2013, 1996, 62..."
2,they presume their audience wo n't sit still f...,0,"[101, 2027, 3653, 23545, 2037, 4378, 24185, 10..."
3,this is a visually stunning rumination on love...,1,"[101, 2023, 2003, 1037, 17453, 14726, 19379, 1..."


In [ ]:
tokensizer.encode("hii this is hige ano")

[101, 7632, 2072, 2023, 2003, 7632, 3351, 2019, 2080, 102]

In [ ]:
tokensizer.encode("i love it")

[101, 1045, 2293, 2009, 102]

In [ ]:
tokens = tokensizer.tokenize("hii this is hige ano")
print(tokens)


['hi', '##i', 'this', 'is', 'hi', '##ge', 'an', '##o']


In [ ]:
df=df.head(1000) # we are restricting til 1000 as we just learning so no need for suchh heavy dataset

In [ ]:
max_len=0
for i in df.tokenized:
  if len(i)>max_len:
    max_len=len(i)

In [ ]:
max_len

59

In [ ]:
import numpy as np
padded=np.array([i+[0]*(max_len-len(i)) for i in df.tokenized.values])

In [ ]:
padded

array([[  101,  1037, 18385, ...,     0,     0,     0],
       [  101,  4593,  2128, ...,     0,     0,     0],
       [  101,  2027,  3653, ...,     0,     0,     0],
       ...,
       [  101,  2412,  2156, ...,     0,     0,     0],
       [  101,  1996,  2143, ...,     0,     0,     0],
       [  101,  2005,  2087, ...,     0,     0,     0]])

In [ ]:
attention_mask=np.where(padded!=0,1,0)

In [ ]:
attention_mask.shape


(1000, 59)

## encoding all the tokens with Distilbert

In [ ]:
import torch

In [ ]:
input_ids=torch.tensor(padded)

In [ ]:
attention_mask=torch.tensor(attention_mask)

In [ ]:
with torch.no_grad():
  last_hidden_states=model(input_ids, attention_mask=attention_mask)

In [ ]:
last_hidden_states[0].shape

torch.Size([1000, 59, 768])

In [ ]:
features=last_hidden_states[0][:,0,:].numpy()

In [ ]:
features

array([[-0.21593435, -0.14028908,  0.00831075, ..., -0.13694833,
         0.5867005 ,  0.20112714],
       [-0.1726272 , -0.14476156,  0.00223447, ..., -0.1744254 ,
         0.21386439,  0.3719747 ],
       [-0.05063364,  0.07203968, -0.02959733, ..., -0.07148939,
         0.7185241 ,  0.2622547 ],
       ...,
       [ 0.21724361,  0.00382841,  0.043231  , ..., -0.1078225 ,
         0.48804078,  0.11470812],
       [ 0.08489723, -0.01240848,  0.16863231, ..., -0.0161418 ,
         0.47347474,  0.00606489],
       [ 0.04173026, -0.16812408, -0.05905461, ..., -0.15474145,
         0.37545285,  0.17935489]], dtype=float32)

In [ ]:
target=df[1]

In [ ]:
target

,1
0,1
1,0
2,0
3,1
4,1
...,...
995,0
996,0
997,0
998,1


In [ ]:
df

,0,1,tokenized
0,"a stirring , funny and finally transporting re...",1,"[101, 1037, 18385, 1010, 6057, 1998, 2633, 182..."
1,apparently reassembled from the cutting room f...,0,"[101, 4593, 2128, 27241, 23931, 2013, 1996, 62..."
2,they presume their audience wo n't sit still f...,0,"[101, 2027, 3653, 23545, 2037, 4378, 24185, 10..."
3,this is a visually stunning rumination on love...,1,"[101, 2023, 2003, 1037, 17453, 14726, 19379, 1..."
4,jonathan parker 's bartleby should have been t...,1,"[101, 5655, 6262, 1005, 1055, 12075, 2571, 376..."
...,...,...,...
995,instead of letting the laughs come as they may...,0,"[101, 2612, 1997, 5599, 1996, 11680, 2272, 200..."
996,this tuxedo should have been sent back to the ...,0,"[101, 2023, 10722, 19068, 2080, 2323, 2031, 20..."
997,ever see one of those comedies that just seem ...,0,"[101, 2412, 2156, 2028, 1997, 2216, 22092, 200..."
998,the film is impressive for the sights and soun...,1,"[101, 1996, 2143, 2003, 8052, 2005, 1996, 1592..."


# binary class classification using logistic regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(features,target,test_size=0.2,random_state=42)

In [ ]:
clf=LogisticRegression()

In [ ]:
clf.fit(x_train,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_prd=clf.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,y_prd)

0.825